In [20]:
import numpy as np
import meshplot as mp
import results.runs as runs
import os
import matplotlib.pyplot as plt
import torch
import model.sdf_model as sdf_model
import trimesh

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Training curves

In [23]:
results_path

'/Users/ri21540/Documents/PhD/Code/DeepSDF/results/runs/04_11_141928/results.npy'

In [24]:
run_folder = '04_11_141928'
run_dir = os.path.join(os.path.dirname(runs.__file__), run_folder)
results_path = os.path.join(run_dir, 'results.npy')
results = np.load(results_path, allow_pickle=True).item()

train_loss = results['train']['loss']
val_loss = results['val']['loss']
plt.figure(figsize=(9, 6))
plt.title('Results', fontsize=20)
plt.plot(train_loss, label='Train loss')
plt.plot(val_loss, label='Val loss')
plt.xlabel('Epochs', fontsize=20)
plt.ylabel('L1', fontsize=20) 
plt.legend(fontsize=20)
plt.show()

UnpicklingError: STACK_GLOBAL requires str

Extract latent code:

In [9]:
latent_codes = results['train']['latent_codes'][-1]     # last one

# Visualize mesh
First optimised latent code during training:

In [ ]:
latent_code_idx = 0
run_folder = '04_11_134800'
run_dir = os.path.join(os.path.dirname(runs.__file__), run_folder)
weights_path = os.path.join(run_dir, 'weights.pt')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def extract_mesh(data, sdf):
    #sdf_inside = data[(sdf.view(-1)>-0.01) & (sdf.view(-1)<0.01)]
    sdf_inside = data[(sdf.view(-1)<0.01)]
    mesh = trimesh.voxel.ops.points_to_marching_cubes(sdf_inside.detach().cpu().numpy(), pitch=0.015)
    return mesh

model = sdf_model.SDFModelMulti().to(device)
model.load_state_dict(torch.load(weights_path, map_location=torch.device(device)))
model.eval()
grid_values = torch.arange(-1, 1, 0.1) #0.015
grid = torch.meshgrid(grid_values, grid_values, grid_values)
coords = torch.vstack((grid[0].ravel(), grid[1].ravel(), grid[2].ravel())).transpose(1, 0).to(device)

latent_tile = torch.tile(latent_codes[latent_code_idx], (coords.shape[0], 1))
coords_latent = torch.hstack((latent_tile, coords))

with torch.no_grad():
    sdf = model(coords_latent)
mesh = extract_mesh(coords, sdf)
verts = np.asarray(mesh.vertices)
faces = np.asarray(mesh.faces)

In [4]:
shading={'wireframe':False, "colormap": "viridis"}
#mp.plot(sphere_verts, sphere_faces, c=sphere_verts[:, 1], shading=shading)
mp.plot(verts, faces, c=verts[:, 1], shading=shading)

NameError: name 'verts' is not defined